In [1]:
# setup.py
import os
from dotenv import load_dotenv

load_dotenv()

# Set your API keys
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY") 
print(os.getenv("SERPAPI_API_KEY"))


e778a27095ea623bff5c9396c46a50690713e5808e77167f5556b835d50d2062



## What is LangChain?
LangChain is a framework for developing applications powered by large language models (LLMs). LangChain simplifies every stage of the LLM application lifecycle through composable components and third-party integrations.

Core Components Overview
1. Chat Models - The brain of your application
2. Prompts - Instructions to guide the model
3. Chains - Sequential operations
4. Tools - External capabilities
5. Agents - Decision-making systems
6. Memory - Persistent conversation state

1. Chat model setup


In [2]:

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# Initialize the model
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5,
    max_tokens=100
)

# Simple interaction
response = llm.invoke([HumanMessage(content="What is LangChain?")])
print(response.content)


LangChain is an open-source framework designed to simplify the development of applications that utilize large language models (LLMs). It provides a structured way to build applications that can leverage the capabilities of LLMs for various tasks, such as text generation, question answering, summarization, and more.

Key features of LangChain include:

1. **Modularity**: LangChain is built with modular components that can be easily combined and customized. This allows developers to create tailored solutions for their specific use cases.




**2a. Understanding Prompt Templates**

Prompts are the interface between human intent and AI understanding.

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Create a prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful coding assistant specialized in {language}."),
    ("human", "Explain {concept} in {language} with a simple example.")
])

# Initialize model
llm = ChatOpenAI(model="gpt-4o-mini")

# Create a chain
chain = prompt | llm

# Test the chain
result = chain.invoke({
    "language": "Python",
    "concept": "list comprehensions"
})

print(result.content)

List comprehensions in Python provide a concise way to create lists. They are a syntactic sugar that allows you to generate lists in a single line of code, making them more readable and often more efficient than traditional `for` loops.

A basic structure of a list comprehension looks like this:

```python
[expression for item in iterable if condition]
```

- **expression**: This is the value to be added to the new list.
- **item**: The variable that takes the value of the elements in the iterable.
- **iterable**: Any Python iterable (like a list, tuple, or string).
- **condition**: (Optional) A filter that only includes items that satisfy a certain condition.

### Simple Example

Let's say we want to create a list of squares for the numbers from 0 to 9. We can use a list comprehension to do this succinctly:

```python
squares = [x**2 for x in range(10)]
print(squares)
```

### Explanation:

1. `x**2` is the expression that computes the square of `x`.
2. `for x in range(10)` iterates o

**2b. Few Shot Prompting**

In [3]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# Few-shot examples
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "fast", "output": "slow"}
]

# Create few-shot prompt
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "Find the opposite word:"),
    few_shot_prompt,
    ("human", "{input}")
])

# Test it
chain = final_prompt | llm
result = chain.invoke({"input": "bright"})
print(result.content)

dull


### LangChain Expression Language (LCEL)**

**Understanding LCEL**
LCEL takes a declarative approach to building new Runnables from existing Runnables. You describe what should happen, rather than how it should happen, allowing LangChain to optimize the run-time execution.
Key LCEL Concepts

1. Pipe Operator (|): Chains components together
2. RunnableSequence: Sequential execution
3. RunnableParallel: Parallel execution
4. RunnableLambda: Custom functions


#### 3A. LCEL Chain

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Define components
prompt = ChatPromptTemplate.from_template(
    "Write a short {style} story about {topic}"
)

model = ChatOpenAI(model="gpt-4o-mini", max_tokens=500)
output_parser = StrOutputParser()

# Create chain using LCEL
chain = prompt | model | output_parser

# Invoke the chain
result = chain.invoke({
    "style": "mystery",
    "topic": "a missing cat"
})

print(result)

It was a crisp autumn afternoon in the small town of Maplewood when Mrs. Agnes Whittaker burst into the local café, her cheeks flushed and her eyes wide with panic. “Help! My cat, Mr. Whiskers, is missing!” she exclaimed, clutching her oversized handbag as though it were a lifeline.

The café patrons looked up from their steaming cups, the aroma of freshly brewed coffee hanging in the air. Mrs. Whittaker was well-known for her flamboyant hats and her fierce love for her fluffy tabby. Mr. Whiskers had become a fixture in the town, often spotted napping on the sun-drenched windowsills or trailing behind Mrs. Whittaker on her morning walks.

Detective Lena Hart, who had just finished a slice of pie, set her fork down and stood up. “I’ll help you, Mrs. Whittaker,” she offered, her curiosity piqued. “When did you last see him?”

“I let him out into the backyard this morning, but he never came back in!” Mrs. Whittaker’s voice trembled as she spoke. “He always returns for his lunch.”

“Let’s 

#### 3B. Parallel Processing

In [11]:
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda

# Create parallel chains
def analyze_sentiment(text):
    return f"Sentiment: Positive" 

def count_words(text):
    return f"Word count: {len(text.split())}"

def extract_keywords(text):
    return "Keywords: mystery, cat, investigation"

# Parallel processing
analysis_chain = RunnableParallel({
    "sentiment": analyze_sentiment,
    "word_count": RunnableLambda(count_words),
    "keywords": extract_keywords
})

# Test parallel processing
text = "The mysterious case of the missing cat puzzled everyone."
results = analysis_chain.invoke(text)
print(results)

{'sentiment': 'Sentiment: Positive', 'word_count': 'Word count: 9', 'keywords': 'Keywords: mystery, cat, investigation'}


**3C: Complex LCEL Pipeline**

In [17]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel

# Define processing functions
def preprocess_text(inputs):
    # text = inputs["text"].strip().lower()
    return text 

def generate_summary(inputs):
    return f"Summary of: {inputs['processed_text'][:50]}..."

def generate_tags(inputs):
    return ["AI", "LangChain", "Tutorial"]

# Fixed pipeline
pipeline = (
    RunnablePassthrough.assign(
        processed_text=RunnableLambda(preprocess_text)  # This adds processed_text to the dict
    )
    | RunnableParallel({
        "summary": RunnableLambda(generate_summary),
        "tags": RunnableLambda(generate_tags),
        "original": lambda x: x["text"],
        "processed_text": lambda x: x["processed_text"]
    })
)

# Test the pipeline
result = pipeline.invoke({
    "text": "LangChain is a powerful framework for building LLM applications."
})

print("Fixed Result:")
print(result)

# Let's also debug the data flow to understand what's happening
print("\n" + "="*50)
print("DEBUG: Understanding the data flow")
print("="*50)

# Step 1: See what RunnablePassthrough.assign produces
step1 = RunnablePassthrough.assign(
    processed_text=RunnableLambda(preprocess_text)
)

intermediate_result = step1.invoke({
    "text": "LangChain is a powerful framework for building LLM applications."
})

print("After RunnablePassthrough.assign:")
print(intermediate_result)
print("\nNow the parallel functions can access both 'text' and 'processed_text' keys!")

Fixed Result:
{'summary': 'Summary of: The mysterious case of the missing cat puzzled eve...', 'tags': ['AI', 'LangChain', 'Tutorial'], 'original': 'LangChain is a powerful framework for building LLM applications.', 'processed_text': 'The mysterious case of the missing cat puzzled everyone.'}

DEBUG: Understanding the data flow
After RunnablePassthrough.assign:
{'text': 'LangChain is a powerful framework for building LLM applications.', 'processed_text': 'The mysterious case of the missing cat puzzled everyone.'}

Now the parallel functions can access both 'text' and 'processed_text' keys!


### Memory and State Management (15 minutes)
#### Understanding Memory in LangChain

Memory allows your applications to remember previous interactions.

In [18]:
# exercise_4_memory.py
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# Store for chat histories
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Create a simple chatbot with memory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Remember the conversation context."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

chain = prompt | llm

# Add memory to the chain
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# Test conversation with memory
config = {"configurable": {"session_id": "user123"}}

response1 = chain_with_history.invoke(
    {"input": "My name is Alex and I'm learning LangChain"},
    config=config
)
print("Response 1:", response1.content)

response2 = chain_with_history.invoke(
    {"input": "What's my name?"},
    config=config
)
print("Response 2:", response2.content)

Response 1: Hi Alex! That's great to hear! LangChain is a powerful framework for building applications with language models. What specific aspects of LangChain are you interested in learning about?
Response 2: Your name is Alex.


## Module 5: Tools and External APIs 
#### Understanding Tools
Tools are integral to the functionality of LangChain agents, allowing them to perform a wide range of tasks with precision. These predefined functions can handle both simple and complex operations.

**Understanding Decorator**

In [19]:
def simple_function():
    return "Hello World"

# This is a plain function
result = simple_function()
print(result)  

Hello World


In [26]:
def my_decorator(func):
    def wrapper():
        print("Before function execution")
        result = func()
        print(f"Function returned: {result}") 
        print("After function execution")
        return result
    return wrapper

@my_decorator
def simple_function():
    return "Hello World"

# Now the function is "decorated"
result = simple_function()
result


Before function execution
Function returned: Hello World
After function execution


'Hello World'

**5A. Creating Custom Tools**

In [34]:
from langchain_core.tools import tool
from typing import List
import datetime

@tool
def get_current_time() -> str:
    """Get the current date and time."""
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool
def calculate_percentage(part: float, whole: float) -> str:
    """Calculate what percentage 'part' is of 'whole'."""
    if whole == 0:
        return "Cannot divide by zero"
    percentage = (part / whole) * 100
    return f"{percentage:.2f}%"

@tool
def word_count(text: str) -> int:
    """Count the number of words in a text."""
    return len(text.split())

# Test the tools
print(get_current_time.invoke({}))
print(calculate_percentage.invoke({"part": 25, "whole": 100}))
print(word_count.invoke({"text": "Hello world, this is a test"}))

2025-08-29 14:48:33
25.00%
6


**5B. SerpAPI Integration Setup**

In [33]:

import os
from dotenv import load_dotenv
from langchain_community.utilities import SerpAPIWrapper
from langchain_core.tools import Tool, tool

@tool
def search_tool(query: str) -> str:
    """Search Google for current information about a query."""
    try:
        search = SerpAPIWrapper()
        return search.run(query)
    except Exception as e:
        return f"Search failed: {str(e)}"

# Test the tool
print("\n Testing search tool:")
try:
    result = search_tool.invoke({"query": "latest AI news 2025"})
    print("Search successful!")
    print("Results:", str(result)[:200] + "...")
except Exception as e:
    print(f"Error: {e}")





 Testing search tool:
Search successful!
Results: [{'title': 'Expected growth of data centres drives Nvidia - 28 Aug 2025', 'link': 'https://www.bbc.com/reel/video/p0lzn65h/expected-growth-of-data-centres-drives-nvidia-28-aug-2025', 'source': 'BBC', ...


**5C: Multiple Tool Integration**

In [31]:

import math

@tool
def calculator(expression: str) -> str:
    """Safely evaluate mathematical expressions."""
    try:
        # Only allow basic operations for security
        allowed_chars = set('0123456789+-*/.() ')
        if all(c in allowed_chars for c in expression):
            result = eval(expression)
            return str(result)
        else:
            return "Invalid expression"
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def text_analyzer(text: str) -> dict:
    """Analyze text and return statistics."""
    words = text.split()
    return {
        "word_count": len(words),
        "character_count": len(text),
        "sentence_count": text.count('.') + text.count('!') + text.count('?'),
        "paragraph_count": text.count('\n\n') + 1
    }

# Tool collection
tools = [get_current_time, calculator, text_analyzer, search_tool]

# Display available tools
print("Available Tools:")
for tool in tools:
    print(f"- {tool.name}: {tool.description}")

Available Tools:
- get_current_time: Get the current date and time.
- calculator: Safely evaluate mathematical expressions.
- text_analyzer: Analyze text and return statistics.
- search_tool: Search Google for current information about a query.


### 6. Understanding Agents
Agents involve an LLM making decisions about which Actions to take, taking that Action, seeing an Observation, and repeating that until done. LangChain supports the creation of agents that use LLMs as reasoning engines to determine which actions to take.

**6A. Simple ReAct Agent**

In [35]:

from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate

# Define the ReAct prompt template
template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(template)

# Create the agent
tools = [get_current_time, calculator, search_tool]
agent = create_react_agent(llm, tools, prompt)

# Create agent executor
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    max_iterations=5
)

# Test the agent
try:
    result = agent_executor.invoke({
        "input": "What time is it and what's 25% of 200?"
    })
    print("\nFinal Result:", result["output"])
except Exception as e:
    print(f"Error: {e}")



> Entering new AgentExecutor chain...
I need to find out the current time and also calculate 25% of 200. I will first get the current time.

Action: get_current_time  
Action Input: None  2025-08-29 14:51:08I have obtained the current time. Now, I will calculate 25% of 200.

Action: calculator  
Action Input: "200 * 0.25"  50.0I have obtained both the current time and the result of the calculation. 

Final Answer: The current time is 2025-08-29 14:51:08, and 25% of 200 is 50.

> Finished chain.

Final Result: The current time is 2025-08-29 14:51:08, and 25% of 200 is 50.


**6b: Search and Summarize Agent**

In [36]:
# Import all necessary modules
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from datetime import datetime

@tool
def summarize_text(text: str) -> str:
    """Summarize a long text into key points."""
    # Create a summarization chain
    summary_prompt = ChatPromptTemplate.from_template(
        "Summarize the following text in 3-5 bullet points:\n\n{text}"
    )
    summary_chain = summary_prompt | llm | StrOutputParser()
    
    try:
        summary = summary_chain.invoke({"text": text[:2000]})  # Limit text length
        return summary
    except Exception as e:
        return f"Could not summarize: {str(e)}"

@tool
def get_current_time() -> str:
    """Get the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")


# Create enhanced tools list
enhanced_tools = [search_tool, summarize_text, get_current_time]

print(f"Tools available: {[tool.name for tool in enhanced_tools]}")

# Create the prompt for the agent
prompt_template = """You are a helpful assistant with access to tools. Use the tools to help answer questions.

You have access to the following tools:
{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}
{agent_scratchpad}"""

prompt = PromptTemplate.from_template(prompt_template)

# Create enhanced agent
try:
    enhanced_agent = create_react_agent(llm, enhanced_tools, prompt)
    enhanced_executor = AgentExecutor(
        agent=enhanced_agent, 
        tools=enhanced_tools, 
        verbose=True,
        max_iterations=8,
        handle_parsing_errors=True  # This helps with parsing errors
    )
    
    print("✅ Agent created successfully!")
    
    # Test search and summarize
    print("\n🔄 Testing agent...")
    result = enhanced_executor.invoke({
        "input": "Search for information about LangChain 2025 updates and summarize the key points"
    })
    
    print("\n" + "="*50)
    print("FINAL SUMMARY:")
    print("="*50)
    print(result["output"])
    
except Exception as e:
    print(f" Error: {e}")
    print("\nDebugging info:")
    print(f"LLM type: {type(llm)}")
    print(f"Tools: {[tool.name for tool in enhanced_tools]}")
    print(f"Tools types: {[type(tool) for tool in enhanced_tools]}")

Tools available: ['search_tool', 'summarize_text', 'get_current_time']
✅ Agent created successfully!

🔄 Testing agent...


> Entering new AgentExecutor chain...
Thought: I need to find current information about LangChain updates for 2025 and then summarize the key points from that information.  
Action: search_tool  
Action Input: "LangChain 2025 updates"  ['LangChain Changelog ; Revision queueing in LangGraph Platform · August 27, 2025 ; LangSmith Self-Hosted v0.11 · August 18, 2025 ; Trace Mode in LangGraph Studio.', "For those who've used LangChain extensively, what are its current strengths and weaknesses? Has the library improved significantly over the past ...", 'LangChain Changelog ; Node-level caching in LangGraph · May 29, 2025 ; Deferred nodes in LangGraph · May 20, 2025 ; MCP with streamable HTTP transport · May 8, 2025.', 'LangGraph Platform GA: Deploy & manage long-running, stateful agents · May 16, 2025 ; LangGraph Studio v2: Run and debug production traces locally.', "80

## 7. Complete Application - OpenAI + SerpAPI Integration (20 minutes)
Final Project: Research Assistant Agent

In [44]:
import json
from datetime import datetime
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool, StructuredTool
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SerpAPIWrapper
from pydantic import BaseModel, Field

class ResearchAssistant:
    def __init__(self):
        self.llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
        self.search = SerpAPIWrapper()
        self.research_history = []
        
    def web_search(self, query: str) -> str:
        """Search the web for current information on any topic."""
        try:
            results = self.search.run(query)
            return results[:1500]  # Limit result length
        except Exception as e:
            return f"Search failed: {str(e)}"
    
    def save_research(self, topic: str, findings: str) -> str:
        """Save research findings for later reference."""
        research_entry = {
            "timestamp": datetime.now().isoformat(),
            "topic": topic,
            "findings": findings
        }
        self.research_history.append(research_entry)
        return f"Research saved for topic: {topic}"
    
    def get_research_history(self, topic: str = "") -> str:
        """Retrieve previous research on a topic."""
        if not self.research_history:
            return "No previous research found."
        
        if topic:
            relevant = [r for r in self.research_history if topic.lower() in r['topic'].lower()]
            if relevant:
                return f"Previous research on {topic}: {relevant[-1]['findings']}"
            return f"No previous research found for {topic}"
        
        return f"Total research entries: {len(self.research_history)}"
    
    def create_agent(self):
        # Create tools using StructuredTool.from_function for better multi-argument support
        @tool
        def web_search_tool(query: str) -> str:
            """Search the web for current information on any topic."""
            return self.web_search(query)
        
        class SaveResearchInput(BaseModel):
            """Input schema for saving research findings."""
            topic: str = Field(description="The research topic or subject")
            findings: str = Field(description="The research findings or summary to save")
        
        def save_research_impl(topic: str, findings: str) -> str:
            """Implementation function for saving research."""
            return self.save_research(topic, findings)
        
        save_research_tool = StructuredTool.from_function(
            func=save_research_impl,
            name="save_research",
            description="Save research findings for later reference. Provide both topic and findings.",
            args_schema=SaveResearchInput
        )
        
        @tool
        def get_research_history_tool(topic: str = "") -> str:
            """Retrieve previous research on a topic. Leave topic empty to see all research."""
            return self.get_research_history(topic)
        
        tools = [web_search_tool, save_research_tool, get_research_history_tool]
        
        # Use tool-calling agent instead of ReAct agent for better multi-argument tool support
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a research assistant. Help users find information and keep track of research.
            
When saving research with the save_research tool, always provide both topic and findings as separate arguments.
Be thorough in your research and always save important findings for future reference."""),
            ("placeholder", "{chat_history}"),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}")
        ])
        
        agent = create_tool_calling_agent(self.llm, tools, prompt)
        
        return AgentExecutor(
            agent=agent,
            tools=tools,
            verbose=True,
            max_iterations=8,
            handle_parsing_errors=True
        )

# Initialize and test the research assistant
assistant = ResearchAssistant()
agent = assistant.create_agent()

# Test scenarios
test_queries = [
    "Research the latest developments in quantum computing in 2025 and save the findings",
    "What did you find about quantum computing earlier?",
    "Search for recent AI breakthroughs and summarize the top 3"
]

print("🔬 RESEARCH ASSISTANT DEMO")
print("="*50)

for i, query in enumerate(test_queries, 1):
    print(f"\n📋 Query {i}: {query}")
    print("-" * 30)
    
    try:
        result = agent.invoke({"input": query})
        print(f"✅ Result: {result['output']}")
    except Exception as e:
        print(f"❌ Error: {e}")
    
    print("\n" + "="*50)

🔬 RESEARCH ASSISTANT DEMO

📋 Query 1: Research the latest developments in quantum computing in 2025 and save the findings
------------------------------


> Entering new AgentExecutor chain...

Invoking: `web_search_tool` with `{'query': 'latest developments in quantum computing 2025'}`


['Explore the latest advancements in quantum computing, sensing, and communication with our comprehensive Quantum Technology Monitor 2025.', 'More experiments with logical qubits · More specialized hardware/software (as opposed to universal quantum computing) · More networking noisy ...', 'They plan rack-mounted modular systems like Forte Enterprise for datacenters and introduce Tempo in 2025 with barium qubits.', 'Build practical, high-impact hybrid applications. · Invest in strategic skilling and access to reliable quantum computers for experimentation.', "The “Quantum Index Report 2025” charts the technology's momentum, with a comprehensive, data-driven assessment of the state of quantum ...", 'The